# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI. Отдельно проанализируйте 3 ошибочно кластеризованных текста (если такие есть).

In [ ]:
%pip install wikipedia

In [ ]:
!pip install nltk

In [ ]:
import wikipedia
import numpy as np
import re
import nltk
nltk.download('punkt_tab')
from nltk import sent_tokenize

In [ ]:
def wiki_downloader(language: str, num_pages: int=10):
    wikipedia.set_lang(language)
    cur_texts = []
    while len(cur_texts) < num_pages:
        text = []
        try:
            name = wikipedia.random(1)
            page = wikipedia.page(name)
        except:
            continue
        else:
            new_text = page.title + '.\n' + page.content
            new_text = re.sub(r'== .*? ==', '', new_text)
            new_text = re.sub(r'[\\d+?]', '', new_text)  #сноски
            new_text = re.sub(r'\s+', ' ', new_text)
            new_text = re.sub(r'http\S+', '', new_text)
            new_text = re.sub(r'\(\s*[а-яa-z]+\.\s*[^)]*\)', '', new_text) #констр. типа (нем. ....)
            new_text = re.sub(r'//.*', '', new_text)  #ссылки на литературу(не очень работает)
            new_text = re.sub(r'[0-9]', '', new_text)  #цифры (с ними хуже)
            sentences = sent_tokenize(new_text)

            for sent in sentences:
                if len(sent) > 20:
                    text.append((sent.strip(), language))
            cur_texts.append(text)

    return cur_texts

In [ ]:
all_texts = []
lang_list = ['kk', 'sr', 'tr', 'de', 'nl']
for l in lang_list:
    all_texts.extend(wiki_downloader(l))

print(*all_texts, sep='\n')

In [ ]:
dataset = {'kk': [], 'sr': [], 'tr': [], 'de' : [], 'nl' : []}
for text in all_texts:
    for sent in text:
        dataset[sent[1]].append(sent[0])


print(dataset)

In [ ]:
!pip install scikit-learn
!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

In [ ]:
all_texts = []
true_labels = []
langs = list(dataset.keys())

for lang_idx, language in enumerate(langs):
    sentences = dataset[language]
    for sent in sentences:
        all_texts.append(sent)
        true_labels.append(lang_idx)

X = np.zeros((len(all_texts), 768))

for i, text in enumerate(all_texts):
    X[i] = embed(text)

cluster = KMeans(5)
cluster.fit(X)
labels = np.array(cluster.labels_)

ari_score = adjusted_rand_score(true_labels, labels)

print(f"ARI score: {ari_score}")

Лучший результат: ARI score: 0.949820692722476

In [ ]:
#смотрим, где есть несовпадения
for i in range(len(all_texts)):
   print(all_texts[i], true_labels[i], labels[i])

Метки наши получились такие:

Исходник: kk-0, cr-1, tr-2, de-3, nl-4

При кластеризации: kk-2, cr-3, tr-0, de-1, nl-4

Неправильные предложения:

1) Potsamer Platz Leipziger Platz (kuzey yarısına) Stresemannstraße Erna-Berger-Straße Schwartzkopffstraße/Pflugstraße, evlerin arka bahçesine. 2 1 >>>>> здесь, очевидно, большая часть текста на немецком, хотя предложение из турецкого текста, что и повлекло ошибку.

2) William Biassi Nyakwe stan von  bis Ene  beim Yaanarbon FC in Myanmar unter Vertrag. 3 4  >>>>> Упомянуты иноязычные имя, название ФК, страна с нетипичным для немецкого языка сочетанием согласных.

3) Sebastiani wer in  cantor in e katheraal van Königsberg en iene als hof-Kapellmeister van  tot . 4 1  >>>>> употреблены слова, которые совпадают в немецком и нидерландском

# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор в разных предложениях). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

In [ ]:
!wget https://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2

In [ ]:
import bz2

with bz2.open('annot.opcorpora.no_ambig_strict.xml.bz2', 'rb') as f_in, open('annot.opcorpora.no_ambig_strict.xml', 'wb') as f_out:
    f_out.write(f_in.read())

In [ ]:
%pip install lxml

In [ ]:
from lxml import etree

In [ ]:
open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

In [ ]:
from collections import defaultdict, Counter

In [ ]:
morpho_dict = defaultdict(Counter)

for sentence in open_corpora.xpath('//tokens'):
    for token in sentence.xpath('token'):
        word = token.xpath('@text')[0]
        gram_info = token.xpath('tfr/v/l/g/@v')
        morpho_dict[word.lower()][tuple(gram_info)] += 1

In [ ]:
#словоформы с самых большим количеством вариантов грамматических разборов (ТОП-10)
morpho_lens = [(word, len(counter)) for word, counter in morpho_dict.items()]
morpho_lens_sorted = sorted(morpho_lens, key=lambda x: x[1], reverse=True)

for word, n_variants in morpho_lens_sorted[:10]:
    print(word, n_variants, morpho_dict[word])

In [ ]:
#самые частотные морфологически омонимичные словоформы (ТОП-20)
morpho_polysems = [(word, sum(counter.values())) for word, counter in morpho_dict.items() if len(counter) > 1]
morpho_polysems_sorted = sorted(morpho_polysems, key=lambda x: x[1], reverse=True)

for word, freq in morpho_polysems_sorted[:20]:
    print(word, freq, morpho_dict[word])

## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуры через DependencyGraph, выберите 3 любых отношения и для каждого найдите топ-5 самых встречаемых пар слов, связанных этим отношением.

Для самой частотной пары слов в каждом из отношений вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)` и сортируя результат по порядку слов в предложениях, аналогично тому как я делал с summaries только у вас будет два слова)
В итоге у вас должен получится что-то такое:

```
### отношение
relation_name

### топ 5 пар слов связанных этим отношением
(word1, word2), (word3, word4), (word5, word6), (word7, word8), (word9, word10)

### подзависимые для самого частотного
(subword word1 subword, word2 subword subword)

... (и так три раза)
```


In [ ]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu

In [ ]:
with open("ru_syntagrus-ud-test.conllu", encoding="utf-8") as f:
    conllu_data = f.read()

In [ ]:
from nltk.parse import DependencyGraph
from collections import Counter

In [ ]:
trees = []

for sent in conllu_data.strip().split('\n\n'):  # убираем лишние пустые блоки
    parsed_sent = sent.strip().split('\n')
    tree = [line for line in parsed_sent if line and line[0] != '#']  # проверка line на пустоту
    if tree:  # добавляем только непустые деревья
        trees.append('\n'.join(tree))

In [ ]:
modifiers_1 = Counter()
modifiers_2 = Counter()
modifiers_3 = Counter()

for tree in trees:
    try:
        d = DependencyGraph(tree)
        d.root = d.nodes[0]
        triples = list(d.triples())
        for e1, rel, e2 in triples:
            if rel == 'nmod':
                modifiers_1[(e1[0], e2[0])] += 1
            elif rel == 'amod':
                modifiers_2[(e1[0], e2[0])] += 1
            elif rel == 'nsubj':
                modifiers_3[(e1[0], e2[0])] += 1
    except AssertionError as e:
            print(f"Пропускаем некорректное дерево")
            continue

In [ ]:
for mod in modifiers_1.most_common(5):
    print(mod[0])

print()

for mod in modifiers_2.most_common(5):
    print(mod[0])

print()

for mod in modifiers_3.most_common(5):
    print(mod[0])

In [ ]:
def get_subtree(nodes, node):


    if not nodes[node]['deps']:
        return [node]

    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps']
                         if rel != 'punct'  # пунктуацию доставать не будем
                         for dep in nodes[node]['deps'][rel]]


In [ ]:
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

In [ ]:
def subtree_finder(word_1, word_2, trees):
    summaries_1 = []
    summaries_2 = []

    for tree in trees:
        try:
            d = DependencyGraph(tree)
            words = [node.get('word') for node_id, node in d.nodes.items()]
            if word_1 in words and word_2 in words:

                for node_i, node in d.nodes.items():
                    if node['word'] == word_1:
                        subtree_ind_1 = flatten(get_subtree(d.nodes, node_i))
                        subtree_ind_sorted_1 = sorted(subtree_ind_1)
                        subtree_words_1 = [d.nodes[idx]['word'] for idx in subtree_ind_sorted_1]
                        summaries_1.append(' '.join(subtree_words_1))

                    elif node['word'] == word_2:
                        subtree_ind_2 = flatten(get_subtree(d.nodes, node_i))
                        subtree_ind_sorted_2 = sorted(subtree_ind_2)
                        subtree_words_2 = [d.nodes[idx]['word'] for idx in subtree_ind_sorted_2]
                        summaries_2.append(' '.join(subtree_words_2))

        except Exception as e:
            continue
    return summaries_1, summaries_2

In [ ]:
summaries_1, summaries_2 = subtree_finder('чемпионата', 'мира', trees)
for i in range(len(summaries_1)):
    print(summaries_1[i], '-', summaries_2[i])

print()


summaries_3, summaries_4 = subtree_finder('время', 'последнее', trees)
for i in range(len(summaries_3)):
    print(summaries_3[i], '-', summaries_4[i])


print()


summaries_5, summaries_6 = subtree_finder('сказал', 'я', trees)
for i in range(len(summaries_5)):
    print(summaries_5[i], '-', summaries_6[i])